In [2]:
import hd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import importlib

In [3]:
c = tf.constant(0.2, dtype=tf.float64)
n_points = 1024

In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [6]:
# We're just calculating the unison, fourth, fifth, octave
vectors = hd.vectors.space_graph(5, 5, bounds=(3.0, 4.0))
vecs = sess.run(vectors)
ratios = np.sort(np.prod(hd.PRIMES[:vectors.shape[-1]] ** vecs, axis=1))
# Print out the logs of the vectors
print("Ratios: \t", ratios)
print("Log-2 Ratios: \t", np.log2(ratios))

Ratios: 	 [ 8.          8.0006876   8.00081633 ... 15.98011364 15.99862491
 16.        ]
Log-2 Ratios: 	 [3.         3.00012399 3.00014721 ... 3.99820576 3.99987601 4.        ]


In [7]:
vecs.shape

(3543, 5)

In [8]:
# Tensor for the permutations of unison, fourth, fifth, octave
perms = hd.cartesian.permutations(np.vstack(vecs), times=2)
sess.run(perms)

array([[[-5., -5., -5.,  4.,  5.],
        [-5., -5., -5.,  4.,  5.]],

       [[-5., -5., -5.,  4.,  5.],
        [-5., -5., -5.,  5.,  4.]],

       [[-5., -5., -5.,  4.,  5.],
        [-5., -5., -4.,  3.,  5.]],

       ...,

       [[ 5.,  5.,  5., -5., -2.],
        [ 5.,  5.,  3., -1., -4.]],

       [[ 5.,  5.,  5., -5., -2.],
        [ 5.,  5.,  4., -3., -3.]],

       [[ 5.,  5.,  5., -5., -2.],
        [ 5.,  5.,  5., -5., -2.]]])

In [10]:
pds = hd.tenney.pd_aggregate_graph(perms)
hds = hd.tenney.hd_aggregate_graph(perms) + 1.0

In [11]:
with tf.variable_scope("default", reuse=tf.AUTO_REUSE):
    log_pitches = tf.get_variable("log_pitches", [n_points, 2], dtype=tf.float64)
    init_op = tf.global_variables_initializer()

In [17]:
xs = np.linspace(0.0, 1.0, n_points)

In [13]:
def z_calc(pds, hds, log_pitches, a=1.0, b=1.0):
    """
    pds: Pitch distance coordinate values of each vector in the space
    hds: Aggregate harmonic distance values of each vector in the space
    log_pitches: The set of pitches to evaluate
    """
    distances = tf.map_fn(lambda x: hd.utilities.reduce_parabola(pds - x, a=a, b=b), log_pitches)
    return tf.reduce_min(distances * hds + hds, axis=1)

In [14]:
def assign_coords_and_return(slice_level, a=1.0, b=1.0):
    """
    Given a y-level (in a 2-dimensional space), take a slice of it with a xs.
    """
    xy_coords = np.hstack([xs[:, None], np.full_like(xs, slice_level)[:, None]])
    z_op = z_calc(pds, hds, tf.constant(xy_coords, dtype=tf.float64), a=a, b=b)
    zs = sess.run(z_op)
    plt.ylim(0.0, 30.0)
    for x in sess.run(pds):
        plt.axvline(x=x[0])
    plt.plot(xs, zs)

In [ ]:
assign_coords_and_return(np.log2(3.0), a=C, b=C)

## 2-dimensional optimization

Setting out the batch sizes, etc.

We're reusing the vectors etc. from above

In [18]:
C = 0.02
LEARNING_RATE = 1.0e-3
CONVERGENCE_THRESHOLD = 1.0e-8
MAX_ITERS = 1000000
DIMENSIONS = 2
BATCH_SIZE = 512
POINTS_PER_DIMENSION = 64
xs = np.linspace(0.00, 1.0, POINTS_PER_DIMENSION)
ys = np.linspace(0.00, 1.0, POINTS_PER_DIMENSION)
xv, yv = np.meshgrid(xs, ys, sparse=False)
zv = np.array([])

In [56]:
with tf.variable_scope("2d_optimization", reuse=tf.AUTO_REUSE):
    log_pitches = tf.get_variable("log_pitches_64x512", [BATCH_SIZE, DIMENSIONS], dtype=tf.float64)

In [57]:
sess.run(tf.global_variables_initializer())

In [58]:
loss = z_calc(pds, hds, log_pitches, a=C, b=C)

In [59]:
stopping_op = hd.optimize.stopping_op(loss, [log_pitches], lr=LEARNING_RATE, ct=CONVERGENCE_THRESHOLD)

In [60]:
# Creates a vector of pairs of shape [n_points**dimensions, dimensions]
starting_coordinates = np.array([xv, yv]).reshape(DIMENSIONS, POINTS_PER_DIMENSION**DIMENSIONS).T
starting_dataset = tf.data.Dataset.from_tensor_slices({
    "coords": tf.constant(starting_coordinates)
})
starting_iterator = starting_dataset.batch(BATCH_SIZE).make_one_shot_iterator()
next_element = starting_iterator.get_next()
assign_op = log_pitches.assign(next_element['coords'])

In [61]:
all_possible_pitches_log = np.empty([0, 2])

In [62]:
while True:
    try:
        sess.run([assign_op])
        for idx in range(MAX_ITERS):
            if (sess.run(stopping_op)):
                print("Converged at iteration: ", idx)
                out_pitches = np.array(sess.run(log_pitches))
                all_possible_pitches_log = np.concatenate([all_possible_pitches_log, out_pitches])
                break
    except tf.errors.OutOfRangeError:
        break

Converged at iteration:  184
Converged at iteration:  209
Converged at iteration:  216
Converged at iteration:  211
Converged at iteration:  199
Converged at iteration:  207
Converged at iteration:  201
Converged at iteration:  187


In [63]:
all_possible_pitches_log

array([[0.00000000e+00, 0.00000000e+00],
       [1.87200106e-20, 0.00000000e+00],
       [3.74400212e-20, 0.00000000e+00],
       ...,
       [1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00]])

In [64]:
print(all_possible_pitches_log.shape)
log_vectors = hd.vectors.pd_graph(vectors)

print(all_possible_pitches_log)
diffs_to_poles = tf.abs(tf.tile(log_vectors[:, None, None], [1, 1, 2]) - all_possible_pitches_log)
mins = tf.argmin(diffs_to_poles, axis=0)
winner = tf.map_fn(lambda m: tf.map_fn(lambda v: vectors[v], m, dtype=tf.float64), mins, dtype=tf.float64)
winners = sess.run(winner)
print(winners)

def vector_to_ratio(vector):
    primes = hd.PRIMES[:vector.shape[0]]
    num = np.where(vector > 0, vector, np.zeros_like(primes))
    den = np.where(vector < 0, vector, np.zeros_like(primes))
    return (
        np.product(np.power(primes, num)), 
        np.product(primes ** np.abs(den))
    )

all_possible_pitches = set()

for row in winners:
    all_possible_pitches.add(tuple([vector_to_ratio(r) for r in row]))

print(len(all_possible_pitches))
print(sorted(all_possible_pitches, key=lambda r: (r[0][0] / r[0][1], r[1][0] / r[1][1])))

(4096, 2)
[[0.00000000e+00 0.00000000e+00]
 [1.87200106e-20 0.00000000e+00]
 [3.74400212e-20 0.00000000e+00]
 ...
 [1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00]]
[[[0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[1. 0. 0.]
  [1. 0. 0.]]

 [[1. 0. 0.]
  [1. 0. 0.]]

 [[1. 0. 0.]
  [1. 0. 0.]]]
20
[((1.0, 1.0), (1.0, 1.0)), ((1.0, 1.0), (5.0, 4.0)), ((1.0, 1.0), (4.0, 3.0)), ((1.0, 1.0), (3.0, 2.0)), ((1.0, 1.0), (5.0, 3.0)), ((1.0, 1.0), (2.0, 1.0)), ((5.0, 4.0), (1.0, 1.0)), ((5.0, 4.0), (5.0, 4.0)), ((4.0, 3.0), (1.0, 1.0)), ((4.0, 3.0), (4.0, 3.0)), ((4.0, 3.0), (2.0, 1.0)), ((3.0, 2.0), (1.0, 1.0)), ((3.0, 2.0), (3.0, 2.0)), ((3.0, 2.0), (2.0, 1.0)), ((5.0, 3.0), (1.0, 1.0)), ((5.0, 3.0), (5.0, 3.0)), ((2.0, 1.0), (1.0, 1.0)), ((2.0, 1.0), (4.0, 3.0)), ((2.0, 1.0), (3.0, 2.0)), ((2.0, 1.0), (2.0, 1.0))]
